In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
#from functions import *
import tqdm

In [52]:
dmso_csv_path = '/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/paths_to_channels_creation/paths_dmso_v1v2.csv'

In [53]:
df_dmso = pd.read_csv(dmso_csv_path)

In [54]:
df_dmso.head()

,Unnamed: 0,plate,well,neg_control_type,C1,C2,C3,C4,C5
0,0,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
1,1,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
2,2,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
3,3,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
4,4,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...


In [55]:
df_dmso = df_dmso[df_dmso["neg_control_type"] == '[dmso]']
suspects = df_dmso.plate.unique()[34:40]


In [56]:
suspects

array(['P100990', 'P100991', 'P101016', 'P101017', 'P101018', 'P101019'],
      dtype=object)

In [57]:
df_dmso = df_dmso[df_dmso["plate"].isin(suspects)]

In [59]:
df_dmso["C1"][0]

KeyError: 0

In [43]:
df_dmso.C1.str.contains("/share").value_counts()


True    4896
Name: C1, dtype: int64

In [44]:
df_dmso.C2.str.contains("/share").value_counts()


True    4896
Name: C2, dtype: int64

In [45]:
df_dmso.C3.str.contains("/share").value_counts()


True    4896
Name: C3, dtype: int64

In [46]:
df_dmso.C4.str.contains("/share").value_counts()


True    4896
Name: C4, dtype: int64

In [47]:
df_dmso.C5.str.contains("/share").value_counts()


True    4896
Name: C5, dtype: int64

In [21]:
test= [df_dmso.plate.unique()[0]]

In [22]:
test

['P015076']

In [23]:
dmso_stats = {}

for plate in tqdm.tqdm(test):
    print(plate)
    dmso_stats[plate] = {}
    rows = df_dmso[df_dmso.plate == plate]
    for c in tqdm.tqdm(['C1','C2','C3','C4','C5']):
        im = []
        
        for i in range(len(rows)):
            path = rows[c].iloc[i]
            im.append(cv2.imread(path, -1))
        
        dmso_stats[plate][c] = {'m': np.mean(im), 'std':np.std(im)}

  0%|          | 0/1 [00:00<?, ?it/s]

P015076



100%|██████████| 1/1 [01:26<00:00, 86.11s/it]


In [24]:
dmso_stats

{'P015076': {'C1': {'m': 298.56553513638613, 'std': 280.17719870419074},
  'C2': {'m': 1182.599314422301, 'std': 1628.216043824049},
  'C3': {'m': 1740.3659357495999, 'std': 846.2971853421564},
  'C4': {'m': 1364.0816585652478, 'std': 1029.1825133612128},
  'C5': {'m': 682.3957518471755, 'std': 704.7482234564715}}}

In [25]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in dmso_stats.items() for innerKey, values in innerDict.items()}

In [26]:
df_test = pd.DataFrame(reform)

In [27]:
df_test.head()

P015076                                                   
             C1           C2           C3           C4          C5
m    298.565535  1182.599314  1740.365936  1364.081659  682.395752
std  280.177199  1628.216044   846.297185  1029.182513  704.748223

In [28]:
df_test.columns


MultiIndex([('P015076', 'C1'),
            ('P015076', 'C2'),
            ('P015076', 'C3'),
            ('P015076', 'C4'),
            ('P015076', 'C5')],
           )

In [29]:
df_test['P015076']['C1']['m']

298.56553513638613

In [30]:
df_test.to_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/paths_to_channels_creation/dmso_stats_v1v2.csv')

In [31]:
df_dmso

,Unnamed: 0,plate,well,neg_control_type,C1,C2,C3,C4,C5
0,0,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
1,1,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
2,2,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
3,3,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
4,4,P015076,B23,[dmso],/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...
...,...,...,...,...,...,...,...,...,...
21595,15547,P101033,O23,[dmso],/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...
21596,15548,P101033,O23,[dmso],/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...
21597,15549,P101033,O23,[dmso],/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...
21598,15550,P101033,O23,[dmso],/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...,/share/data/external-datasets/compoundcenter/s...


In [33]:
df_dmso.isnull().values.any()

False